# ETL - H&M

## Imports

In [1]:
# importando as bibliotecas
import requests
import pandas as pd
import numpy as np
import re
import sqlite3

from datetime import datetime
from bs4 import BeautifulSoup
from sqlalchemy import create_engine

## Data collection - VITRINE

In [2]:
# paginação em HTML
url = 'https://www2.hm.com/en_us/men/products/jeans.html'

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

page = requests.get( url, headers=headers)

soup = BeautifulSoup( page.text, 'html.parser')

# pegando o total items que tem a vitrine
total_item = soup.find_all('h2', class_='load-more-heading')[0].get('data-total')

# descobrindo o numero de paginas
# o comando np.ceil arrendonda para o numero interio acima
page_number = np.ceil( int(total_item) / 36 )

# montando o url do total de produtos da vitrine
url2 = url + '?page-size=' + str( int(page_number*36))


# ======================== COLETAR DADOS NA VITRINE ====================================
# requests irá pegar as informações da pagina html é salvar numa variavel
#url = 'https://www2.hm.com/en_us/men/products/jeans.html'

# o headers é para burlar o html fingindo que é um browser e não python que está captando as informações
# link dos headers de diversos browser : https://www.useragentstring.com/pages/useragentstring.php
# headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

page = requests.get( url2, headers=headers)

# toda a informação da pagina html está na variavel soup
soup = BeautifulSoup(page.text, 'html.parser')

# salvar em uma variavel apenas as informações da vitrine 
# observando a pagina HTML no site (inspecionar)

# comando .find - pega a primeira informação
products = soup.find('ul', class_='products-listing small')

# o comando find_all retorna as informações como lista
product_list = products.find_all('article', class_='hm-product-item')

# colunas da simulação que necessito ter para meu projeto
# id | product_category | product_name | price | color | composition

# coluna id 
product_id = [p.get('data-articlecode') for p in product_list]

# coluna product_category
product_category = [p.get('data-category') for p in product_list]

# coluna product_name
# para pegar as informações do name
# seleciono outra tag dentro d products
product_list = products.find_all('a', class_='link')

# para pegar apenas os nomes utilizo o comando get_text
product_name = [p.get_text('') for p in product_list]

# para pegar os preço de cada calça devo selecionar outra tag
product_list = products.find_all('span', class_='price regular')
price = [p.get_text() for p in product_list]

# juntar as colunas em um dataframe utilizando Pandas
data = pd.DataFrame( [product_id, product_category, product_name, price]).T
data.columns = ['product_id', 'product_category', 'product_name','price']

# cada scrapy é importante colocar datar
data['scrapy_datetime'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

data.head()

,product_id,product_category,product_name,price,scrapy_datetime
0,1024256001,men_jeans_slim,Slim Jeans,$ 19.99,2022-09-16 15:56:13
1,1024256006,men_jeans_slim,Slim Jeans,$ 19.99,2022-09-16 15:56:13
2,0993887007,men_jeans_joggers,Hybrid Regular Denim Joggers,$ 39.99,2022-09-16 15:56:13
3,0938875007,men_jeans_slim,Slim Tapered Jeans,$ 39.99,2022-09-16 15:56:13
4,1004199007,men_jeans_skinny,Skinny Cropped Jeans,$ 29.99,2022-09-16 15:56:13


## Data collection by Product

In [3]:
# empty dataframe
df_compositions = pd.DataFrame()


for i in range(len(data)):
    # API requests

    # url do unico produto
    url = 'https://www2.hm.com/en_us/productpage.' + data.loc[i, 'product_id'] + '.html'
#     url = 'https://www2.hm.com/en_us/productpage.' + '1024256001' + '.html'

    page = requests.get( url, headers=headers)

    #BeautifulSoup object
    soup = BeautifulSoup( page.text, 'html.parser')

    # =================================COLOR NAME ===============================================================

    # seleciona a tag a com a class filter.. para pegar(get) a class que tem a informa desejada -- que é a cor
    product_list = soup.find_all('a', class_='filter-option miniature active') + soup.find_all('a', class_='filter-option miniature')
    color_name = [p.get('data-color') for p in product_list]

    # pegando cada id da cor do produto
    product_id = [p.get('data-articlecode') for p in product_list]


    # montando um dataframe
    df_color = pd.DataFrame([product_id, color_name]).T
    df_color.columns = ['product_id', 'color_name']
    
    for j in range(len(df_color)):
        # API requests

        # url do unico produto
        url = 'https://www2.hm.com/en_us/productpage.' + df_color.loc[j, 'product_id'] + '.html'

        page = requests.get( url, headers=headers)

        #BeautifulSoup object
        soup = BeautifulSoup( page.text, 'html.parser')
        
        #======================================== Product Name ============================
        product_name = soup.find_all('hm-product-name')
        product_name = product_name[0].get_text().replace('\n', '')
        
        #======================================== Product Price ============================
        product_price = soup.find_all('div',class_ = 'primary-row product-item-price')
        product_price = re.findall(r'\d+\.\d+', product_price[0].get_text() )[0]
        product_price
        
        # ================================= COMPOSITION ===============================================================
        # pegando as informações dos detalhes da calça
        product_composition_list = soup.find_all('div',class_="details-attributes-list-item")
        product_composition = [list(filter(None, p.get_text().split('\n'))) for p in product_composition_list]

        # fazendo o dataframe da lista de detalhes da calça
        df_aux = pd.DataFrame(product_composition).T
        df_aux.columns = df_aux.iloc[0]

        # deletar a primeira linha
        df_aux = df_aux[1:]

        # selecionar apenas as colunas desejadas
        df_composition = df_aux[['Fit', 'Composition', 'Art. No.']]
        #excluindo apenas as linhas onde todos os valores estão vazios
        df_composition = df_composition.dropna(how='all',axis=0)

        # preenchendo os valores vazios
        df_composition = df_composition.fillna( method='ffill')

        # remover pocket lining, shell and lining
        df_composition['Composition'] = df_composition['Composition'].replace('Pocket lining: ', '', regex=True)
        df_composition['Composition'] = df_composition['Composition'].replace('Shell: ', '', regex=True)
        df_composition['Composition'] = df_composition['Composition'].replace('Lining: ', '', regex=True)
        df_composition['Composition'] = df_composition['Composition'].replace('Pocket: ', '', regex=True)
        
        # rename columns
        df_composition.columns = ['fit', 'composition', 'product_id']
        
        # recebendo novas colunas
        df_composition['product_name'] = product_name
        df_composition['product_price'] = product_price

        # ================================= JUNTANDO AS INFORMAÇÕES ==============================================================

        df_composition = pd.merge(df_composition, df_color, how='left', on='product_id')

        # concat todos os dataframes
        df_compositions = pd.concat( [df_compositions, df_composition], axis=0 )


# gerando style id + color id
# df_color

df_compositions['style_id'] = df_compositions['product_id'].apply(lambda x: x[:-3])
df_compositions['color_id'] = df_compositions['product_id'].apply(lambda x: x[-3:])

# cada scrapy é importante colocar datar
df_compositions['scrapy_datetime'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

## Data Cleaning

In [4]:
# product id
df = df_compositions.dropna(subset=['product_id'])

df['product_name'] = df['product_name'].apply(lambda x: x.replace(' ', '_').lower())

df['product_price'] = df['product_price'].astype(float)

df['fit'] = df['fit'].apply(lambda x: x.replace(' ', '_').lower() if pd.notnull(x) else x )

df['color_name'] = df['color_name'].apply(lambda x: x.replace(' ', '_').lower() if pd.notnull(x) else x )


# ================= composition =================================
# quebrar a coluna composition por virgula
# expand me retorna como dataframe
# TENHO UM DATAFRAME COM OS NOMES SEPARADOS, MAS FORAM DE ORDEM DE COLUNA
# O INDEX ESTÁ CORRETO
df1 = df['composition'].str.split(',', expand=True).reset_index(drop=True)

# primeira coisa
# definir quais as colunas que eu quero
### df['Composition'].unique()
# cotton | spandex | polyester |
# criando um dataframe de referencia das colunas que eu quero
# esse dataframe tem que ter o mesmo tamanho de linhas do dataframe que eu quero concatenar
df_ref = pd.DataFrame( index=np.arange( len( df ) ), columns=['cotton', 'spandex', 'polyester'] )

########## cotton
# criar um dataframe de uma unica coluna, ou seja uma series
# o df1[0] e df1[1] contem apenas os cottons
df_cotton_0 = df1.loc[df1[0].str.contains('Cotton', na=True), 0]
df_cotton_0.name = 'cotton'

df_cotton_1 = df1.loc[df1[1].str.contains('Cotton', na= True), 1]
df_cotton_1.name = 'cotton'

# combine
df_cotton = df_cotton_0.combine_first(df_cotton_1)

# # a concateneação é atraves do index
df_ref = pd.concat( [df_ref, df_cotton], axis=1 )
# # ecluir colunas duplicadas (deixando a ultima [last])
df_ref = df_ref.iloc[:,~df_ref.columns.duplicated(keep='last')]

# ########### spandex
# # o df1[1] e [2] contem spandex
df_spandex_1 = df1.loc[df1[1].str.contains('Spandex', na=True),1]
df_spandex_1.name = 'spandex'

df_spandex_2 = df1.loc[df1[2].str.contains('Spandex', na= True), 2]
df_spandex_2.name = 'spandex'

# combine
df_spandex = df_spandex_1.combine_first(df_spandex_2)

# # a concateneação é atraves do index
df_ref = pd.concat( [df_ref, df_spandex], axis=1 )
# # ecluir colunas duplicadas (deixando a ultima [last])
df_ref = df_ref.iloc[:,~df_ref.columns.duplicated(keep='last')]


# #polyester
# # o df1[0] e df1[1] contem apenas os cottons
df_polyester_0 = df1.loc[df1[0].str.contains('Polyester', na=True),0]
df_polyester_0.name = 'polyester'

df_polyester_1 = df1.loc[df1[1].str.contains('Polyester', na= True), 1]
df_polyester_1.name = 'polyester'

df_polyester = df_polyester_0.combine_first( df_polyester_1 )

# # a concateneação é atraves do index
df_ref = pd.concat( [df_ref, df_polyester], axis=1 )
# # ecluir colunas duplicadas (deixando a ultima [last])
df_ref = df_ref.iloc[:,~df_ref.columns.duplicated(keep='last')]

# join of combine with product_id
df_aux = pd.concat( [df['product_id'].reset_index(drop=True), df_ref], axis=1 )

# pegando apenas os número das composições
df_aux['cotton'] = df_aux['cotton'].apply( lambda x: int(re.search('\d+',x).group(0)) / 100 if pd.notnull(x) else x)
df_aux['spandex'] = df_aux['spandex'].apply( lambda x: int(re.search('\d+',x).group(0)) / 100 if pd.notnull(x) else x)
df_aux['polyester'] = df_aux['polyester'].apply( lambda x: int(re.search('\d+',x).group(0)) / 100 if pd.notnull(x) else x)

# # final join
df_aux = df_aux.groupby('product_id').max().reset_index().fillna(0)
df = pd.merge( df, df_aux, on='product_id', how='left' )


# drop columns 
df = df.drop( columns= ['composition'])

# drop duplicates
df = df.drop_duplicates()

## Data Insert

In [5]:
# mudando as ordens da coluna
data_insert = df[[
    'product_id',
    'style_id',
    'color_id',
    'product_name',
    'product_price',
    'fit',
    'color_name',
    'cotton',
    'spandex',
    'polyester',
    'scrapy_datetime'
]]

In [6]:
# # criando uma query de tabela
# query_showroom_schema = """
#     CREATE TABLE vitrine(
#     product_id,
#     style_id,
#     color_id,
#     product_name,
#     product_price,
#     fit,
#     color_name,
#     cotton,
#     spandex,
#     polyester,
#     scrapy_datetime
#     )
# """

In [7]:
# create table
conn = sqlite3.connect('database_hm.sqlite3')
cursor = conn.execute( query_showroom_schema)
conn.commit()

In [8]:
# create database connection
conn = create_engine('sqlite:///database_hm.sqlite', echo=False)

#data insert
data_insert.to_sql('vitrine', con=conn, if_exists='append', index=False)

154